In [2]:
import os as os
import ctypes
import pickle
import multiprocessing as mp

import tqdm
import numpy as np
from scipy.io import loadmat
from scipy.signal import detrend
import nibabel as nib

import popeye.utilities as utils
from popeye import css
from popeye.visual_stimulus import VisualStimulus, resample_stimulus

In [3]:
f = open('css_results.pkl', 'rb')
results = pickle.load(f)

In [4]:
results

 ...]

In [5]:
len(results)

218854

In [6]:
output = [r for r in results if r is not None]

In [7]:
output

 ...]

In [8]:
len(output)

218854

In [9]:
nii = nib.load('/Users/aditya/Documents/GitHub/prf-mapping/sample_data/JC/RF1/JC_RF1_vista/bar_seq_1_ss5.nii.gz')

In [10]:
nii

In [11]:
dims = list(nii.shape)

In [12]:
dims

[128, 128, 128, 201]

In [13]:
dims = dims[0:3]

In [14]:
dims

[128, 128, 128]

In [15]:
dims.append(8)

In [16]:
estimates = np.zeros(dims)

In [17]:
estimates.shape

(128, 128, 128, 8)

In [18]:
import math

In [30]:
t = 0

In [32]:
for fit in output:
    if not np.isnan(fit.rsquared):
        voxel_dat = list(fit.estimate)
        reshape_voxel_dat = []
        reshape_voxel_dat.append(math.atan2(voxel_dat[1], voxel_dat[0]))
        reshape_voxel_dat.append(fit.rsquared)
        reshape_voxel_dat.append(math.sqrt( voxel_dat[0] ** 2 + voxel_dat[1] ** 2))
        reshape_voxel_dat.append(voxel_dat[2])
        reshape_voxel_dat.append(voxel_dat[3])
        reshape_voxel_dat.append(voxel_dat[0])
        reshape_voxel_dat.append(voxel_dat[1])
        reshape_voxel_dat.append(voxel_dat[4])
        t += 1
        # print(reshape_voxel_dat)
        # print(math.atan2(voxel_dat[1], voxel_dat[0]))
        # print(math.sqrt( voxel_dat[0] ** 2 + voxel_dat[1] ** 2 ))
        # voxel_dat.append(fit.rsquared)
        reshape_voxel_dat = np.array(reshape_voxel_dat)
        # print(fit.voxel_index)
        # print(voxel_dat)
        estimates[tuple(fit.voxel_index)] = reshape_voxel_dat

In [33]:
t 

64408

In [20]:
grid_parent = nii

In [21]:
aff = grid_parent._affine
hdr = grid_parent._header
hdr.set_data_shape(dims)

In [22]:
estimates

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [23]:
estimates.shape

(128, 128, 128, 8)

In [21]:
import nibabel

In [22]:
nifti_estimates = nibabel.Nifti1Image(estimates,aff,header=hdr)

In [24]:
nib.save(nifti_estimates, "output/JC/prf_css.nii.gz")